# Lab 4: Urban Heat Island Detection using Thermal Satellite Imagery

## Goal
In this exercise, you will:
1. Select and compare two thermal satellite images from Kraków (or another city that has recently undergone intensive, thoughtless concrete development) – one from **2013** and one from **2024**.
2. Each image must be acquired on a **hot summer day** (T > 27°C based on IMGW meteorological data).
3. Images must have **low cloud cover** (< 20%).
4. Visualize thermal data and analyze surface temperature differences (Urban Heat Island effect).

## Task 1: Area of Interest (AOI)
- Define an AOI over the Reduta Street area in Kraków.
- Use `ee.Geometry.Polygon` or `ee.Geometry.Rectangle`.

In [14]:
import ee

# Initialize the Earth Engine API
ee.Initialize()

# Define the AOI using ee.Geometry.Rectangle
aoi = ee.Geometry.Rectangle([19.88, 50.010, 19.92, 50.030])  # Replace with actual coordinates

# Print the AOI to verify
print('AOI:', aoi.getInfo())

AOI: {'type': 'Polygon', 'coordinates': [[[19.88, 50.01], [19.92, 50.01], [19.92, 50.03], [19.88, 50.03], [19.88, 50.01]]]}


## Task 2: Download and Analyze IMGW Meteorological Data
- Visit: [IMGW Archive](https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/)
- Navigate to:
  - `dane_meteorologiczne/dobowe/synop/2013/`
  - `dane_meteorologiczne/dobowe/synop/2024/`
- Download `s_d_tmax.csv` for both years.
- Filter the rows for **station ID 12566 (e.g. Kraków-Balice)**.
- Identify days in **July or August** with **TMAX > 27°C**.

## Task 3: Select Landsat 8 Images Matching These Dates
- In Earth Engine, use `LANDSAT/LC08/C02/T1_L2` collection.
- Apply filters:
  - `.filterBounds(aoi)`
  - `.filterDate()` for the matching day
  - `.filterMetadata('CLOUD_COVER', 'less_than', 20)`
- Try to find **one image from 2013** and **one image from 2024** that match your hot days list.

In [20]:
# Define the Landsat 8 image collection
landsat_collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')

# Filter for 2013 image
image_2013 = landsat_collection.filterBounds(aoi) \
    .filterDate('2013-08-01', '2013-08-31') \
    .filterMetadata('CLOUD_COVER', 'less_than', 20) \
    .first()


landsat_collection9 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')

# Filter for 2024 image
image_2024 = landsat_collection9.filterBounds(aoi) \
    .filterDate('2024-08-01', '2024-08-31') \
    .filterMetadata('CLOUD_COVER', 'less_than', 20) \
    .first()

# Get and print only the date of the selected images
print('2013 Image Date:', image_2013.date().format('YYYY-MM-dd').getInfo())
print('2024 Image Date:', image_2024.date().format('YYYY-MM-dd').getInfo())

2013 Image Date: 2013-08-07
2024 Image Date: 2024-08-13


## Task 4: Process Thermal Band (ST_B10)
- Convert Band 10 to Brightness Temperature (Kelvin):
  `TB = ST_B10 * 0.00341802 + 149.0`
- Create a visualization of each image using the same color scale.

In [21]:
import geemap 
# Extract the thermal band (ST_B10) for both images
thermal_band_2013 = image_2013.select('ST_B10')
thermal_band_2024 = image_2024.select('ST_B10')

# Convert to Brightness Temperature (Kelvin)
tb_2013 = thermal_band_2013.multiply(0.00341802).add(149.0)
tb_2024 = thermal_band_2024.multiply(0.00341802).add(149.0)

# Define visualization parameters
vis_params = {
    'min': 290,  # Adjust based on expected temperature range
    'max': 320,  # Adjust based on expected temperature range
    'palette': ['blue', 'green', 'yellow', 'red']
}

# Add layers to the map for visualization
Map = geemap.Map()  # Assuming geemap is being used for visualization
Map.centerObject(aoi, 13)
Map.addLayer(tb_2013, vis_params, 'Temperature 2013')
Map.addLayer(tb_2024, vis_params, 'Temperature 2024')
Map.addLayer(aoi, {}, 'AOI')

Map

Map(center=[50.02000102479521, 19.899999999993007], controls=(WidgetControl(options=['position', 'transparent_…

## Task 5: Compare and Interpret
- Compare the two maps.
- Optionally calculate difference: `TB_2024 - TB_2013`
- Discuss: did the surface temperature increase in the area?
- Is there evidence of an Urban Heat Island effect related to development?

In [ ]:
#